In [1]:
from src.data.load import DataLoader
from pathlib import Path

loader = DataLoader('data')
loader.get_data()

Getting iowa_licor_sales dataset
Dataset iowa_licor_sales already present

Getting wallmart_sales dataset
Dataset wallmart_sales already present

Getting supermarket_sales dataset
Dataset supermarket_sales already present

Getting superstore_sales dataset
Dataset superstore_sales already present

Getting lifetime_value dataset
Dataset lifetime_value already present

Getting corona dataset
Dataset corona already present

Getting climate_change_earth_temperature dataset
Dataset climate_change_earth_temperature already present

Getting nifty50_stock_market dataset
Dataset nifty50_stock_market already present



In [2]:
import polars as pl

z = loader.load_dataset("corona")
z.head()

/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/corona/covid_19_data.csv


SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
i64,str,str,str,str,f64,f64,f64
1,"""01/22/2020""","""Anhui""","""Mainland China""","""1/22/2020 17:00""",1.0,0.0,0.0
2,"""01/22/2020""","""Beijing""","""Mainland China""","""1/22/2020 17:00""",14.0,0.0,0.0
3,"""01/22/2020""","""Chongqing""","""Mainland China""","""1/22/2020 17:00""",6.0,0.0,0.0
4,"""01/22/2020""","""Fujian""","""Mainland China""","""1/22/2020 17:00""",1.0,0.0,0.0
5,"""01/22/2020""","""Gansu""","""Mainland China""","""1/22/2020 17:00""",0.0,0.0,0.0


In [ ]:
#%pip install kaggle polars-lts-cpu pyarrow pystan

In [3]:
# EarthTemperatureFormatter().fit_transform(loader.load_dataset("climate_change_earth_temperature").head())
loader.load_dataset("corona").head()

/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/corona/covid_19_data.csv


SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
i64,str,str,str,str,f64,f64,f64
1,"""01/22/2020""","""Anhui""","""Mainland China""","""1/22/2020 17:00""",1.0,0.0,0.0
2,"""01/22/2020""","""Beijing""","""Mainland China""","""1/22/2020 17:00""",14.0,0.0,0.0
3,"""01/22/2020""","""Chongqing""","""Mainland China""","""1/22/2020 17:00""",6.0,0.0,0.0
4,"""01/22/2020""","""Fujian""","""Mainland China""","""1/22/2020 17:00""",1.0,0.0,0.0
5,"""01/22/2020""","""Gansu""","""Mainland China""","""1/22/2020 17:00""",0.0,0.0,0.0


In [4]:
from src.data.data_formatter import (
    SupermarketSalesFormatter,
    IowaLicorSalesFormatter,
    WallmartSalesFormatter,
    SuperstoreSalesFormatter,
    LifetimeValueFormatter,
    EarthTemperatureFormatter,
    Nifty50StockMarketFormatter,
    CoronaFormatter,
    )

dataset_formatters = {
    'supermarket_sales': SupermarketSalesFormatter(),
    'iowa_licor_sales': IowaLicorSalesFormatter(),
    'wallmart_sales': WallmartSalesFormatter(),
    'superstore_sales': SuperstoreSalesFormatter(),
    'lifetime_value': LifetimeValueFormatter(),
    'climate_change_earth_temperature': EarthTemperatureFormatter(),
    'nifty50_stock_market': Nifty50StockMarketFormatter(),
    'corona': CoronaFormatter(),
    }

# Test if all are running
for dataset_name, formatter in dataset_formatters.items():
    formatter.fit_transform(loader.load_dataset(dataset_name).head())

/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/supermarket_sales/supermarket_sales - Sheet1.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/iowa_licor_sales/Iowa_Liquor_Sales.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/wallmart_sales/Walmart.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/superstore_sales/train.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/lifetime_value/test.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/climate_change_earth_temperature/GlobalLandTemperaturesByCountry.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/nifty50_stock_market/NIFTY50_all.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/corona/covid_19_data.csv


In [27]:
from src.data.random_experiment import RandomConstantLiftExperiment
setup = RandomConstantLiftExperiment("lifetime_value", loader.load_dataset("lifetime_value"),
        eligible_treatment_period=[0.5, 0.5],
        lift_limit=1)
treated_data = setup.get_treated_data()
treated_data.head()

/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/lifetime_value/test.csv


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

country_segment,STV,product,join_date,credit_card_level,product_type,user_id
str,f64,str,datetime[μs],str,str,f64
"""Other Countries""",8.25,"""product_1""",2019-07-01 00:00:00,"""standard""","""type_ex""",1.188741e6
"""US""",8.25,"""product_2""",2019-07-01 00:00:00,"""standard""","""type_ex""",1.188768e6
"""RU""",8.25,"""product_1""",2019-07-01 00:00:00,"""standard""","""type_ex""",1.188785e6
"""AU""",8.25,"""product_4""",2019-07-01 00:00:00,"""standard""","""type_ex""",1.1888e6
"""US""",8.25,"""product_4""",2019-07-01 00:00:00,"""standard""","""type_ex""",1.18885e6


In [45]:
from src.models.preprocessing import SyntheticControlPreProcessing

synth_preprocessing = SyntheticControlPreProcessing(
    # SupermarketSalesFormatter(),
    LifetimeValueFormatter(),
    setup.experiment_setup, 
)

processed_data = synth_preprocessing.fit_transform(treated_data)
processed_data.head()

date,Other Countries,target,ZA,RU,TH,CL,CA,GB,IL
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64
2019-07-23 00:00:00,3005.829675,28272.865926,200.818875,439.474875,77.733,18.265125,681.84255,514.5943,265.831875
2019-07-27 00:00:00,3465.26635,33799.433052,127.58325,534.802875,80.340375,106.71075,882.14955,913.854875,825.125625
2019-07-17 00:00:00,3459.797925,22503.809973,101.563125,576.008625,12.6225,28.502625,399.46025,307.91305,667.0125
2019-07-26 00:00:00,3991.9334,30380.695939,143.958375,651.7338,19.855125,54.697875,708.7278,776.171675,315.06225
2019-07-13 00:00:00,3164.3358,18693.0741,262.001625,680.538075,3.5775,97.150875,757.57635,628.56255,320.849625


In [46]:
print(setup.experiment_setup.lift_size)
print(setup.experiment_setup.target_col)
print(setup.experiment_setup.treated_groups)
print(setup.experiment_setup.treatment_start_date)
print(setup.experiment_setup.treatment_end_date)

0.8616187413773878
STV
['LK', 'NZ', 'MY', 'AU', 'US', 'SG', 'NG']
2019-07-16 00:00:00
2019-07-31 00:00:00


In [29]:
processed_data.write_csv('~/Documents/ltv_test.csv')

In [30]:
from src.data.random_experiment import RandomConstantLiftExperiment
setup = RandomConstantLiftExperiment("supermarket_sales", loader.load_dataset("supermarket_sales"))

for dataset_name in [
    "lifetime_value", 
    # "climate_change_earth_temperature",
    # "nifty50_stock_market", 
    # "corona"
    ]:
    setup = RandomConstantLiftExperiment(
        dataset_name,
        loader.load_dataset(dataset_name),
        eligible_treatment_period=[0.5, 0.5],
        lift_limit=1
    )
    treated_data = setup.get_treated_data()

    synth_preprocessing = SyntheticControlPreProcessing(
        setup.data_formatter,
        setup.experiment_setup, 
    )

    preprocessed_data = synth_preprocessing.fit_transform(treated_data)
    print(preprocessed_data.head())

/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/supermarket_sales/supermarket_sales - Sheet1.csv
/Users/raphaeltamaki/Documents/personal_git/causal_methodologies_analysis/data/lifetime_value/test.csv
shape: (5, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ date      ┆ CL        ┆ CA        ┆ IL        ┆ … ┆ RU        ┆ Other     ┆ TH        ┆ target   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ Countries ┆ ---       ┆ ---      │
│ datetime[ ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ ---       ┆ f64       ┆ f64      │
│ μs]       ┆           ┆           ┆           ┆   ┆           ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2019-07-1 ┆ 28.502625 ┆ 399.46025 ┆ 667.0125  ┆ … ┆ 576.00862 ┆ 3459.7979 ┆ 12.6225   ┆ 22503.80 │
│ 7         ┆           ┆           ┆          

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [ ]:
preprocessed_data

# Test

In [ ]:
# nest_asyncio is needed for running pystan in a notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
setup.experiment_setup.treatment_start_date

In [ ]:
pandas_data = (
    preprocessed_data
    .filter(pl.col("date") < pl.lit(setup.experiment_setup.treatment_start_date))
    .to_pandas()
    )
feat_cols = [col for col in pandas_data.columns if col not in (setup.experiment_setup.treated_groups + ["date", "target"])]

In [ ]:
import numpy as np
import stan
# import stan.model

# Define the model
model_code = """
        data {
            int<lower=1> N; // number of observations
            int<lower=1> cols; // number of features
            vector[N] y; // target variable
            matrix[N, cols] X; // design matrix
        }
        parameters {
            vector[cols] coefs; // coefficients
            real<lower=0.0001> sigma;
            real bias;
        }
        model {
            sigma ~ cauchy(0, 2);
            coefs ~ double_exponential(0, 0.5);
            y ~ normal(X * coefs + bias, sigma);
        }
        """

# Define the data for the model
y = np.array(pandas_data['target'].fillna(0))
X = np.matrix(pandas_data[feat_cols].fillna(0))
cols = X.shape[1]
is_treated = list((pandas_data["date"] >= setup.experiment_setup.treatment_start_date).astype(int))
N = X.shape[0]
stan_data = {"X": X, "y": y, "N": N, "cols": cols, "is_treated": is_treated}
# # Fit the model to the data
posterior = stan.build(model_code, data=stan_data)

In [ ]:
fit = posterior.sample(num_chains=3, num_samples=1000)

In [ ]:
import seaborn as sns
# sns.histplot(fit['treatment_coef'][0])
coef = fit['coefs'].mean(axis=1)
[c for c in coef if abs(c) > 0.1]

In [ ]:
coef_cols = [f'coefs.{i+1}' for i in range(len(feat_cols))]
coef_samples = np.matrix(fit.to_frame()[coef_cols]).transpose()


In [ ]:
post_period_data = (
    preprocessed_data
    .filter(pl.col("date") >= pl.lit(setup.experiment_setup.treatment_start_date))
    .select(feat_cols)
    .to_pandas()
    )
prediction = np.matmul(post_period_data, coef_samples) + fit['bias']
prediction

In [ ]:
post_period_target = (
    preprocessed_data
    .filter(pl.col("date") >= pl.lit(setup.experiment_setup.treatment_start_date))
    .select("target")
    .to_pandas()
    )
result = prediction.apply(lambda x: post_period_target["target"] - x)
result

In [ ]:
result.mean(axis=0).std()

In [1]:
import arviz as az

import causalpy as cp


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/raphaeltamaki/opt/anaconda3/envs/pydata_2024/lib/python3.12/site-packages/causalpy/pymc_models.py:171: SyntaxWarning: invalid escape sequence '\s'
  """
/Users/raphaeltamaki/opt/anaconda3/envs/pydata_2024/lib/python3.12/site-packages/causalpy/pymc_models.py:224: SyntaxWarning: invalid escape sequence '\s'
  """
/Users/raphaeltamaki/opt/anaconda3/envs/pydata_2024/lib/python3.12/site-packages/causalpy/pymc_models.py:419: SyntaxWarning: invalid escape sequence '\s'
  """


KeyboardInterrupt: 

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
seed = 42

In [ ]:
df = cp.load_data("sc")
treatment_time = 70

In [ ]:
# Note, we do not want an intercept in this model
result = cp.pymc_experiments.SyntheticControl(
    df,
    treatment_time,
    formula="actual ~ 0 + a + b + c + d + e + f + g",
    model=cp.pymc_models.WeightedSumFitter(
        sample_kwargs={"target_accept": 0.95, "random_seed": seed}
    ),
)

In [ ]:
fig, ax = result.plot(plot_predictors=True)

In [ ]:
result.summary()

In [ ]:
az.summary(result.post_impact.mean("obs_ind"))